In [ ]:
# --- 0. Configuración Inicial y Carga de Librerías ---
import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, pipeline
from PIL import Image
import numpy as np
import os
import gradio as gr
import requests
import fitz # PyMuPDF for PDF handling
from pdf2image import convert_from_path # For converting PDF pages to images

# --- 0.1 Configuración de PaliGemma ---
PALI_GEMMA_MODEL_ID = "google/paligemma-3b-mix-224" # O "google/paligemma-10b-mix-448" si tienes más VRAM
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16 if torch.cuda.is_available() else torch.float32

# Cargar el procesador y el modelo PaliGemma UNA SOLA VEZ
print(f"Cargando modelo PaliGemma: {PALI_GEMMA_MODEL_ID} en {DEVICE} con dtype {DTYPE}...")
try:
    processor = AutoProcessor.from_pretrained(PALI_GEMMA_MODEL_ID)
    pali_gemma_model = PaliGemmaForConditionalGeneration.from_pretrained(
        PALI_GEMMA_MODEL_ID,
        torch_dtype=DTYPE,
        device_map=DEVICE,
        # load_in_8bit=True # Descomentar si tienes problemas de VRAM en T4
    ).eval()
    print("Modelo PaliGemma cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo PaliGemma: {e}")
    print("Asegúrate de haber aceptado los términos en Hugging Face y de tener suficiente VRAM.")
    # Considera salir o manejar el error de forma más robusta en un entorno de producción
    pali_gemma_model = None
    processor = None

# --- 0.2 Configuración del Modelo de Traducción ---
# Usaremos un modelo de Transformers para traducción
# "Helsinki-NLP/opus-mt-en-es" para Inglés a Español
# "Helsinki-NLP/opus-mt-es-en" para Español a Inglés
print("Cargando modelos de traducción...")
try:
    translator_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es", device=0 if DEVICE == "cuda" else -1)
    translator_es_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en", device=0 if DEVICE == "cuda" else -1)
    print("Modelos de traducción cargados exitosamente.")
except Exception as e:
    print(f"Error al cargar modelos de traducción: {e}")
    translator_en_es = None
    translator_es_en = None

# --- Variables Globales para la Sesión ---
current_image = None # Almacenará la imagen PIL cargada por el usuario
indexed_text_content = "" # Almacenará el texto extraído de la imagen (OCR)

# --- 0.3 Funciones Auxiliares ---

def generate_paligemma_response(pil_image, prompt, max_new_tokens=500):
    """Genera una respuesta de texto de PaliGemma."""
    if pali_gemma_model is None or processor is None:
        return "Error: Modelo PaliGemma no cargado."
    try:
        inputs = processor(text=prompt, images=pil_image, return_tensors="pt").to(DEVICE, DTYPE)
        with torch.no_grad():
            generated_ids = pali_gemma_model.generate(**inputs, max_new_tokens=max_new_tokens)
        response_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        response_text = response_text.replace(prompt, "").strip() # Limpia el prompt de la respuesta
        return response_text
    except Exception as e:
        return f"Error al procesar la imagen con PaliGemma: {e}. Intenta con una imagen más pequeña o un prompt diferente."

def perform_ocr_with_paligemma(pil_image):
    """Realiza OCR/Transcripción usando PaliGemma."""
    prompt = "Transcribe todo el texto presente en esta imagen."
    return generate_paligemma_response(pil_image, prompt, max_new_tokens=1500) # Más tokens para transcripción

def explain_image_content(pil_image):
    """Pide a PaliGemma que explique el contenido de la imagen."""
    prompt = "Describe detalladamente lo que se ve en esta imagen, centrándote en el contenido educativo o informativo."
    return generate_paligemma_response(pil_image, prompt, max_new_tokens=1000)

def translate_text(text_to_translate, source_lang, target_lang):
    """Traduce texto usando modelos Hugging Face."""
    if not text_to_translate:
        return "No hay texto para traducir."
    if source_lang == target_lang:
        return text_to_translate

    try:
        if source_lang == "en" and target_lang == "es" and translator_en_es:
            res = translator_en_es(text_to_translate)
            return res[0]['translation_text']
        elif source_lang == "es" and target_lang == "en" and translator_es_en:
            res = translator_es_en(text_to_translate)
            return res[0]['translation_text']
        else:
            return "Modelos de traducción no disponibles para la dirección seleccionada o no cargados."
    except Exception as e:
        return f"Error durante la traducción: {e}"

# --- Funciones de la Interfaz Gradio ---

def index_data(file):
    """
    Simula la carga y extracción de contenido de una imagen o PDF.
    Esta función se llama cuando el usuario sube un archivo.
    """
    global current_image
    global indexed_text_content

    indexed_text_content = "" # Resetear contenido indexado
    current_image = None

    if file is None:
        return "Por favor, sube un archivo.", None, "", ""

    file_path = file.name # Gradio pasa el objeto archivo, su path está en .name

    if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
        print(f"Procesando imagen: {file_path}")
        pil_image = Image.open(file_path).convert("RGB")
        current_image = pil_image
        status_message = "Imagen cargada. Puedes hacer preguntas o transcribir."

    elif file_path.lower().endswith('.pdf'):
        print(f"Procesando PDF: {file_path}")
        try:
            # Convertir la primera página del PDF a imagen. Podrías iterar si quieres todo el PDF.
            images = convert_from_path(file_path, first_page=1, last_page=1)
            if images:
                pil_image = images[0].convert("RGB")
                current_image = pil_image
                status_message = "PDF (primera página) cargado como imagen. Puedes hacer preguntas o transcribir."
            else:
                status_message = "No se pudieron extraer imágenes del PDF."
                return status_message, None, "", ""
        except Exception as e:
            status_message = f"Error al procesar el PDF: {e}. Asegúrate de que 'poppler-utils' esté instalado."
            return status_message, None, "", ""
    else:
        status_message = "Formato de archivo no soportado. Por favor, sube una imagen (JPG, PNG) o PDF."
        return status_message, None, "", ""

    # Intentar transcribir automáticamente el texto para indexar (si hay imagen válida)
    if current_image:
        print("Realizando transcripción inicial de la imagen para el RAG...")
        indexed_text_content = perform_ocr_with_paligemma(current_image)
        if len(indexed_text_content) > 5000: # Limitar para no saturar prompts
            indexed_text_content = indexed_text_content[:5000] + "..."
        print("Transcripción inicial completada.")
        return status_message, current_image, indexed_text_content, ""
    else:
        return status_message, None, "", ""


def rag_query(query):
    """
    Responde a preguntas sobre la imagen, utilizando el texto indexado si es relevante.
    """
    global current_image
    global indexed_text_content

    if current_image is None:
        return "Por favor, primero sube una imagen o PDF.", ""

    if not query:
        return "Por favor, ingresa una pregunta."

    # Combinar la pregunta del usuario con el texto extraído (indexado) como contexto
    # Esto simula un RAG simple: el "recuperador" es el OCR, y el "generador" es PaliGemma.
    context_prompt = ""
    if indexed_text_content:
        context_prompt = f"Basado en el siguiente texto extraído de la imagen: '{indexed_text_content}' y la imagen, responde a la pregunta."

    full_prompt = f"{context_prompt}\n\nPregunta: {query}\n\nRespuesta:"

    print(f"Prompt enviado a PaliGemma: {full_prompt[:200]}...") # Para depuración

    response = generate_paligemma_response(current_image, full_prompt, max_new_tokens=300)
    return response, indexed_text_content # Retorna la respuesta y el texto indexado


def perform_translation_mode(text_to_translate, source_lang, target_lang):
    """Función para el modo traductor."""
    return translate_text(text_to_translate, source_lang, target_lang)

# --- Interfaz de Gradio ---

with gr.Blocks() as demo:
    gr.Markdown("# Asistente de Estudio Multimodal con PaliGemma y Traducción")
    gr.Markdown("Sube una imagen o PDF (se procesará la primera página) para hacer preguntas, transcribir o traducir su contenido.")

    with gr.Tab("Asistente Visual (Preguntas y OCR)"):
        with gr.Row():
            with gr.Column():
                file_input = gr.File(label="Sube tu material (Imagen/PDF)")
                image_display = gr.Image(label="Imagen del Material", type="pil", show_label=True)
                status_output = gr.Textbox(label="Estado del Archivo", interactive=False)
                indexed_text_display = gr.Textbox(label="Texto Transcrito (OCR) de la Imagen (Para Referencia)", interactive=False, lines=5)
                load_button = gr.Button("Cargar y Analizar Archivo")

            with gr.Column():
                question_input = gr.Textbox(label="Haz una pregunta sobre el material", placeholder="Ej: ¿Qué se explica en este diagrama? o ¿Cuál es la fórmula aquí?")
                rag_output = gr.Textbox(label="Respuesta del Asistente", interactive=False, lines=5)
                query_button = gr.Button("Enviar Pregunta")
                transcribe_button = gr.Button("Transcribir Texto (OCR)")
                explanation_button = gr.Button("Explicar Contenido Visual")
                clear_button = gr.Button("Limpiar Todo")


        load_button.click(
            fn=index_data,
            inputs=file_input,
            outputs=[status_output, image_display, indexed_text_display]
        )
        query_button.click(
            fn=rag_query,
            inputs=[question_input],
            outputs=[rag_output, indexed_text_display]
        )
        transcribe_button.click(
            fn=lambda img: perform_ocr_with_paligemma(img) if img else "Por favor, sube una imagen primero.",
            inputs=image_display,
            outputs=rag_output # Muestra la transcripción en la caja de respuesta
        )
        explanation_button.click(
            fn=lambda img: explain_image_content(img) if img else "Por favor, sube una imagen primero.",
            inputs=image_display,
            outputs=rag_output # Muestra la explicación en la caja de respuesta
        )
        clear_button.click(
            fn=lambda: ("", None, "", "", ""),
            inputs=[],
            outputs=[file_input, image_display, status_output, indexed_text_display, rag_output]
        )

    with gr.Tab("Modo Traductor"):
        gr.Markdown("Introduce texto y selecciona los idiomas para traducir.")
        with gr.Row():
            translation_input = gr.Textbox(label="Texto a Traducir", lines=5)
            with gr.Column():
                source_lang_dropdown = gr.Dropdown(["en", "es"], label="Idioma Origen", value="en")
                target_lang_dropdown = gr.Dropdown(["en", "es"], label="Idioma Destino", value="es")
                translate_button = gr.Button("Traducir")
            translation_output = gr.Textbox(label="Texto Traducido", interactive=False, lines=5)

        translate_button.click(
            fn=perform_translation_mode,
            inputs=[translation_input, source_lang_dropdown, target_lang_dropdown],
            outputs=translation_output
        )

demo.launch(debug=True, share=True) # share=True para un link público en Colab